# Using Knowledge tools

**Step 1**: Install NuGet packages

To get started with Azure AI Foundry, you need to install the required NuGet packages:

In [ ]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7"
#r "nuget: Microsoft.SemanticKernel.Agents.Core"
#r "nuget: DotNetEnv, 3.1.0"
#r "nuget: Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview"

Installed Packages DotNetEnv, 3.1.0 Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7 Microsoft.SemanticKernel, 1.56.0 Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview Microsoft.SemanticKernel.Agents.Core, 1.56.0

**Step 2**: Read environment variables

In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.

In [22]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\ai-foundry\../..\.env


**Step 3**: Instantiate the Project client

Create an instance of the `PersistentAgentsClient` using the endpoint URL and the Azure AI Foundry project.

Note that **DefaultAzureCredential** is used for authentication. To successfully authenticate, log in to the Azure CLI using `az login --use-device-code`.

In [23]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;
using Azure.Identity;

var projectEndpoint = Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_ENDPOINT");
#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
var client = AzureAIAgent.CreateAgentsClient(projectEndpoint, new DefaultAzureCredential());
#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 4**: Define your agent

In this step, we define the attributes for the agent, including its name, instructions, tools (if any), and the model to be used.

After running this code, the agent will be created in the Azure AI Foundry project. 

In [24]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Agents.AzureAI;
using Azure.Identity;
using Azure.AI.Agents.Persistent;

var modelId = Environment.GetEnvironmentVariable("MODEL");
var agentName = "FileSearchAgent";
var agentInstructions = @"You are a helpful assistant that can answer questions about Contoso products and services.";

// Use Path.Combine with Environment.CurrentDirectory for portable relative path resolution
var filePath = Path.Combine(Environment.CurrentDirectory, "../../resources/contoso-bikes.pdf");
var fileStream = File.OpenRead(filePath);

PersistentAgentFileInfo fileInfo = await client.Files.UploadFileAsync(fileStream, PersistentAgentFilePurpose.Agents, "contoso-bikes.pdf");
PersistentAgentsVectorStore fileStore = await client.VectorStores.CreateVectorStoreAsync([fileInfo.Id]);

var agentDefinition = await client.Administration.CreateAgentAsync(
    modelId,
    name: agentName,
    instructions: agentInstructions,
    tools: [new FileSearchToolDefinition()],
    toolResources: new()
    {
        FileSearch = new()
        {
            VectorStoreIds = { fileStore.Id },
        }
    });


**Step 5**: Create the agent

Finally, we create the agent using the `CreateAgentAsync` method of the `PersistentAgentsClient`. This method takes the agent definition and returns the created agent.

In [25]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
       
var agent = new AzureAIAgent(agentDefinition, client);

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 5**: Creat a new thread for the agent

In [26]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
       
var thread = new AzureAIAgentThread(client);

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 6** : Create a new message for the agent

In [ ]:
    #pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

    var invokeAgentAsync = async (AzureAIAgentThread agentThread, string input) =>
    {
        ChatMessageContent message = new(AuthorRole.User, input);
        await foreach (ChatMessageContent response in agent.InvokeAsync(message, agentThread))
        {
            Console.WriteLine($"{response.Role}: {response.Content}");
        }
    };

    await invokeAgentAsync(thread, "Can you suggest bikes for city commuting?");
    #pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

assistant: Here are some suggestions for bikes suitable for city commuting based on Contoso's product catalog:

1. **Contoso City Sprint** - A fast commuter bike designed for urban environments. Price: $799.99【4:0†source】.
2. **Contoso Metro Glide** - A sleek bike ideal for smooth city rides. Price: $699.99【4:0†source】.
3. **Contoso E-City** - An electric bike for convenient city commuting. Price: $1,599.99【4:0†source】.
4. **Contoso City Cruiser** - A comfortable bike for daily city commuting. Price: $499.99【4:2†source】.
5. **Contoso Urban Glide** - Suitable for smooth urban rides, combining style with functionality. Price: $749.99【4:3†source】.
6. **Contoso Retro Glide** - A stylish retro bike perfect for city riding. Price: $799.99【4:4†source】.
7. **Contoso E-Urban** - An electric bike built for fast commutes in urban settings. Price: $1,699.99【4:4†source】.

Feel free to choose one depending on your preferences, like electric assistance, style, or price. Let me know if you need more h

**Step 7**: Create additional messages to the same thread

In [20]:
 await invokeAgentAsync(thread, "Which ones are in stock at the Seattle store?");


assistant: At the Seattle store, the following bikes suitable for city commuting are in stock:

1. **Contoso Urban Sprint**: Stock available - 11 units【8:0†source】.
2. **Contoso City Sprint**: Stock available - 10 units【8:2†source】.
3. **Contoso City Pro**: Stock available - 9 units【8:4†source】.
4. **Contoso Urban Max**: Stock available - 12 units【8:4†source】.

Let me know if you need further details!


**Step 8**: Clean up

Delete the agent and the thread to clean up resources.

In [ ]:
await thread.DeleteAsync();
await client.Administration.DeleteAgentAsync(agent.Id);